In [1]:
#Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from bs4 import BeautifulSoup



In [2]:
# Configurar Selenium
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

# URL del sitio web
url = 'https://ciudad3d.buenosaires.gob.ar/'
driver.get(url)

# Cerrar el cartel si existe
try:
    # Espera a que aparezca el botón de cierre del cartel
    close_button = WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '.reactour__close'))
    )
    
    # Haz clic en el botón de cierre
    close_button.click()
except:
    # Puedes manejar la excepción si el botón de cierre no está presente
    pass

# Esperar 5 segundos antes de enviar la tecla de escape
time.sleep(5)

# Enviar la tecla de escape
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.ESCAPE)

# Encuentra el campo de búsqueda por ID
input_direccion = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'usig-autocomplete-input'))
)

# Ingresa la dirección que deseas buscar (en este caso, Av la plata 2984)
direccion = 'Av la plata 2984'
input_direccion.clear()  # Limpia el campo de búsqueda en caso de que haya texto preexistente
input_direccion.send_keys(direccion)

# Espera a que aparezcan las sugerencias
suggestions_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'usig-autocomplete-menu'))
)

# Asegúrate de que se haya realizado la acción antes de realizar otras interacciones
driver.implicitly_wait(5)

# Aquí puedes intentar hacer clic en el primer elemento de la lista de sugerencias
# Este es solo un ejemplo y puede necesitar ajustes según la estructura real de tu HTML
try:
    suggestion_element = suggestions_box.find_element(By.XPATH, '//div[@role="option"][1]')
    suggestion_element.click()
except Exception as e:
    print(f"No se pudo hacer clic en la sugerencia: {e}")

# Imprime la dirección (en este caso, solo la dirección que se ha ingresado)
print(f'Dirección: {direccion}')

# Espera a que aparezca el botón de "Datos Básicos" en el DOM
datos_basicos_button = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, '//button[h5[text()="Datos Básicos"]]'))
)

# Espera a que el botón sea visible antes de hacer clic en él
WebDriverWait(driver, 20).until(
    EC.visibility_of(datos_basicos_button)
)

# Ahora haz clic en el botón "Datos Básicos" usando JavaScript
try:
    driver.execute_script("arguments[0].click();", datos_basicos_button)
except Exception as e:
    print(f"No se pudo hacer clic en 'Datos Básicos' usando JavaScript: {e}")



# HASTA ACA ANDA PERFECTO


# Espera a que se cargue la información de datos básicos (ajusta el tiempo de espera según sea necesario)
time.sleep(5)

# Obtén el contenido HTML actual de la página
html_content = driver.page_source

# Crea un objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Busca el texto "Superficie de parcela"
superficie_parcela_tag = soup.find('h6', string='Superficie de parcela')


# Accede al texto dentro del tag y elimina los espacios en blanco
valor_numerico_texto = superficie_parcela_tag.find_next('h6', class_='MuiTypography-subtitle2').text.strip()

    # Imprime el valor numérico
print("Superficie lote: ", valor_numerico_texto)


# Encuentra el elemento de la flecha hacia atrás
flecha_atras = driver.find_element(By.XPATH, '//*[@data-testid="ArrowBackIcon"]')


# Haz clic en la flecha hacia atrás
flecha_atras.click()

# Encuentra el botón de "Codigo urbanistico" y haz clic en él
codigo_urbanistico_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[h5[text()="Código Urbanístico"]]'))
)
codigo_urbanistico_button.click()

# PARTE NUEVA

time.sleep(5)

html_content_codigo = driver.page_source

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(html_content_codigo, 'html.parser')

# Buscar la información requerida
unidad_sustentabilidad_tag = soup.find('h6', string='Unidades de Sustentabilidad de Altura Alta')
altura_maxima = unidad_sustentabilidad_tag.find_next('h6').text

# extraigo solo los numeros de la altura maxima, al ser un sitio dinamico se complicaba mucho con scraping
altura_maxima_calculo = re.search(r'Altura Máxima: ([0-9,.]+) metros', altura_maxima).group(1) if re.search(r'Altura Máxima: ([0-9,.]+) metros', altura_maxima) else None

print(unidad_sustentabilidad_tag.text)
print("Altura Máxima:", altura_maxima_calculo)

# Busca el texto "Superficie de parcela"
area_edificaable_planta_texto = soup.find('h6', string='Área edificable en planta')


# Accede al texto dentro del tag y elimina los espacios en blanco
area_edificable_planta = superficie_parcela_tag.find_next('h6', class_='MuiTypography-subtitle2').text.strip()

    # Imprime el valor numérico
print("El area edificable en planta es de: ", area_edificable_planta)


# Cierra el navegador al finalizar
driver.quit()

altura_maxima_calculo = float(altura_maxima_calculo.replace(',', '.'))
area_edificable_planta = float(area_edificable_planta.replace(',', '.').replace(' m²', ''))

altura_estandar = 2.8
espacios_comunes = 0.85

#aca van las incidencias de los barrios
incidencia1 = 135
incidencia2 = 160

#calculo de pre factibilidad

pisos_edificables =int(altura_maxima_calculo / altura_estandar)

pre_factibilidad_metros_totales = (pisos_edificables * area_edificable_planta) * espacios_comunes


print('El valor del terreno es entre: ', pre_factibilidad_metros_totales*incidencia1, ' y ', pre_factibilidad_metros_totales*incidencia2, 'U$S')

Dirección: Av la plata 2984
Superficie lote:  590,2 m²


ElementClickInterceptedException: Message: element click intercepted: Element <svg class="MuiSvgIcon-root MuiSvgIcon-fontSizeSmall css-3x1dg9" focusable="false" aria-hidden="true" viewBox="0 0 24 24" data-testid="ArrowBackIcon">...</svg> is not clickable at point (114, 75). Other element would receive the click: <p class="MuiTypography-root MuiTypography-body1 css-4s6ou3">...</p>
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF715DB2142+3514994]
	(No symbol) [0x00007FF7159D0CE2]
	(No symbol) [0x00007FF7158776AA]
	(No symbol) [0x00007FF7158C88F3]
	(No symbol) [0x00007FF7158C68ED]
	(No symbol) [0x00007FF7158C40EC]
	(No symbol) [0x00007FF7158C2EA5]
	(No symbol) [0x00007FF7158B6A0F]
	(No symbol) [0x00007FF7158E5FEA]
	(No symbol) [0x00007FF7158B63B6]
	(No symbol) [0x00007FF7158E6490]
	(No symbol) [0x00007FF7159028F6]
	(No symbol) [0x00007FF7158E5D93]
	(No symbol) [0x00007FF7158B4BDC]
	(No symbol) [0x00007FF7158B5C64]
	GetHandleVerifier [0x00007FF715DDE16B+36

In [5]:
altura_maxima_calculo = float(altura_maxima_calculo.replace(',', '.'))
area_edificable_planta = float(area_edificable_planta.replace(',', '.').replace(' m²', ''))

altura_estandar = 2.8
espacios_comunes = 0.85

#aca van las incidencias de los barrios
incidencia1 = 135
incidencia2 = 160

#calculo de pre factibilidad

pisos_edificables =int(altura_maxima_calculo / altura_estandar)

pre_factibilidad_metros_totales = (pisos_edificables * area_edificable_planta) * espacios_comunes


print('El valor del terreno es entre: ', pre_factibilidad_metros_totales*incidencia1, ' y ', pre_factibilidad_metros_totales*incidencia2, 'U$S')


El valor del terreno es entre:  541803.6  y  642137.6 U$S
